アルゴリズムの検討

- 上がりそうな銘柄を買いポジションで持つための銘柄選別

In [ ]:
from pathlib import Path
import csv
from typing import List
import datetime

import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import stock

pio.renderers.default = 'notebook'

In [ ]:
code_list_csv = stock.constants.DATA_DIR / "margin_trade_target_list.csv"
daily_data_dir = Path.home() / "remote/gdrive/stock/data/daily"
one_minute_data_dir = Path.home() / "remote/gdrive/stock/data/one_minute"

assert code_list_csv.exists()
assert daily_data_dir.exists()
assert one_minute_data_dir.exists()

with open(code_list_csv) as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader)
    codes = [row[0] for row in csv_reader]

In [ ]:
def get_daily_data(code, data_dir=daily_data_dir) -> List:
    csvs = sorted(data_dir.glob(f"*{code}.csv"))
    if len(csvs) == 0:
        stock.logger.warning(f"No csv files of `{code}` in the directory.")
        return []
    daily_data_csv = csvs[0]
    assert daily_data_csv.exists()
    with open(daily_data_csv) as f:
        csv_reader = csv.reader(f)
        header = next(csv_reader)
        data = [
            [
                datetime.datetime.strptime(row[0], "%Y/%m/%d").timestamp() + 60 * 60 * 9,  # utcで0時になるように変換
                float(row[2]),
                float(row[3]),
                float(row[4]),
                float(row[5]),
                float(row[6])
            ]
            for row in csv_reader
        ]
    return np.array(data)

In [ ]:
data = get_daily_data(codes[0])

In [ ]:
alpha = 1.5
short = 12
long = 26

macd, signal_line, histogram = stock.chart.trend.macd(data[:, 4], alpha=1.5, short=short, long=long)
short_ema = stock.chart.trend.ema(data[:, 4], window_size=short, alpha=alpha)
long_ema = stock.chart.trend.ema(data[:, 4], window_size=long, alpha=alpha)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.6, 0.4], vertical_spacing=0.0)
xdata = np.arange(0, len(data[:, 0]))
fig.add_trace(go.Candlestick(x=xdata, open=data[:, 1], high=data[:, 2], low=data[:, 3], close=data[:, 4]), row=1, col=1)
fig.add_trace(go.Scatter(x=xdata, y=short_ema, name="short_ema"), row=1, col=1)
fig.add_trace(go.Scatter(x=xdata, y=long_ema, name="long_ema"), row=1, col=1)
fig.add_trace(go.Scatter(x=xdata, y=macd, name="macd"), row=2, col=1)
fig.add_trace(go.Scatter(x=xdata, y=signal_line, name="signal_line"), row=2, col=1)
fig.add_trace(go.Bar(x=xdata, y=histogram, name="histogram"), row=2, col=1)

fig.update_yaxes(range=[data[:, 3].min(), data[:, 2].max()], row=1, col=1)
fig.update_layout(xaxis_rangeslider_visible=False, 
                  xaxis2_rangeslider_visible=True,
                  margin=go.layout.Margin(l=5, r=5, t=5, b=5, autoexpand=True),
                  )
#fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x unified")                  
fig.update_traces(xaxis='x2')
                 
fig.show()

In [ ]:
window_sizes = [5, 20]

fig = go.Figure(data=[
    go.Candlestick(
        x = np.arange(0, len(data[:, 0])),
        open=data[:, 1],
        high=data[:, 2],
        low=data[:, 3],
        close=data[:, 4],
        name=""
    )
])

for window_size in window_sizes:
    ma_start = stock.chart.trend.moving_average(data[:, 1], window_size=window_size)
    ma_end = stock.chart.trend.moving_average(data[:, 4], window_size=window_size)
    ma_diff = ma_end[1:] - ma_end[:-1]

    fig.add_trace(go.Scatter(x=np.arange(1 + window_size, len(ma_end) + 1), y=ma_end[window_size:], name=f"{window_size} day moving average"))
fig.show()

In [ ]:
raw_diff = data[:, 4] - data[:, 1]
print("Number of start < end  : {} / {} ({:.2f} %)".format(
    (raw_diff > 0).sum(), len(raw_diff), (raw_diff > 0).sum() / len(raw_diff) * 100
))